In [3]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [4]:
%pwd

'c:\\Users\\Tanveer\\Text-Summarizer\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\Tanveer\\Text-Summarizer'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [10]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum", use_fast=False)


{2025-05-03 00:31:55,640: INFO: config: PyTorch version 2.4.1 available.}


In [ ]:
class DataTransformation:
    def __init__(self, config):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum", use_fast=False)
tokenizer.save_pretrained("./local_tokenizer")



('./local_tokenizer\\tokenizer_config.json',
 './local_tokenizer\\special_tokens_map.json',
 './local_tokenizer\\spiece.model',
 './local_tokenizer\\added_tokens.json')

In [14]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

class DataTransformation:
    def __init__(self, model_name="google/pegasus-cnn_dailymail"):
        self.tokenizer = PegasusTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = PegasusForConditionalGeneration.from_pretrained(model_name)

    def transform(self, text: str) -> str:
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding="longest")
        summary_ids = self.model.generate(inputs["input_ids"], max_length=100, num_beams=4, early_stopping=True)
        return self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example usage:
# transformer = DataTransformation()
# print(transformer.transform("Long article text here..."))


In [17]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

{2025-05-03 00:37:14,227: INFO: common: yaml file: C:\Users\Tanveer\Text-Summarizer\config\config.yaml loaded successfully}
{2025-05-03 00:37:14,231: INFO: common: yaml file: C:\Users\Tanveer\Text-Summarizer\params.yaml loaded successfully}
{2025-05-03 00:37:14,234: INFO: common: Created Directory at: artifacts}
{2025-05-03 00:37:14,238: INFO: common: Created Directory at: artifacts/data_transformation}


TypeError: __init__() got an unexpected keyword argument 'config'